<a href="https://colab.research.google.com/github/Osmanoor/visper/blob/Osmanoor-patch-1/visper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mohammed-elhaj/visper

Cloning into 'visper'...
remote: Enumerating objects: 393, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (218/218), done.
remote: Total 393 (delta 143), reused 152 (delta 69), pack-reused 101
Receiving objects: 100% (393/393), 16.97 MiB | 34.47 MiB/s, done.
Resolving deltas: 100% (147/147), done.


In [1]:
%cd "visper"

/content/visper


In [3]:
# Install PyTorch and other required packages
!pip install torch torchvision torchaudio
!pip install pytorch-lightning
!pip install sentencepiece
!pip install av
!pip install hydra-core --upgrade
!pip install mediapipe
!pip install streamlit

# Install ffmpeg
!apt-get install ffmpeg

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 28.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [2]:
#Book an IP adress for sreamlit
!wget -q -O - ipv4.icanhazip.com


34.83.196.78


In [3]:
%cd   /content/visper/

/content/visper


In [6]:
# Download the model weights (assuming the files are directly accessible via URLs)
!wget -O visper_vsr_base.pth https://huggingface.co/tiiuae/visper/resolve/main/visper_vsr_base.pth?download=true

# Download the dataset and prepare the data
# Assuming the data is a video file for testing
!wget --content-disposition http://www.doc.ic.ac.uk/~pm4115/autoAVSR/autoavsr_demo_video.mp4 -O clip.mp4

--2024-06-20 08:38:34--  https://huggingface.co/tiiuae/visper/resolve/main/visper_vsr_base.pth?download=true
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.114, 3.163.189.37, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/d7/69/d769a85e89ebb579b52c264d1af69408c7ddcfd8c54f0b96305ebdc3bc5b5db5/d6b45e0a9988ae3e747496dd0f24c7b3f5e019358e743df8e66579a38e755608?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27visper_vsr_base.pth%3B+filename%3D%22visper_vsr_base.pth%22%3B&Expires=1719131914&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTEzMTkxNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2Q3LzY5L2Q3NjlhODVlODllYmI1NzliNTJjMjY0ZDFhZjY5NDA4YzdkZGNmZDhjNTRmMGI5NjMwNWViZGMzYmM1YjVkYjUvZDZiNDVlMGE5OTg4YWUzZTc0NzQ5NmRkMGYyNGM3YjNmNWUwMTkzNThlNzQzZGY4

In [4]:
import os
import hydra
import torch
import torchaudio
import torchvision
from lightning_vsr import ModelModule, get_beam_search_decoder
from datamodule.av_dataset import cut_or_pad
from datamodule.transforms import AudioTransform, VideoTransform
from WER.measures import get_wer as wer
from WER.measures import get_cer as cer
import numpy as np
import torch.multiprocessing as mp
import time, re
from tqdm import tqdm
from subprocess import CalledProcessError, run

class InferencePipeline(torch.nn.Module):
    def __init__(self, cfg, detector="mediapipe"):
        super(InferencePipeline, self).__init__()
        self.modality = cfg.data.modality
        self.multi_lang = cfg.data.multi_lang
        if self.modality in ["audio", "audiovisual"]:
            self.audio_transform = AudioTransform(subset="test")
        if self.modality in ["video", "audiovisual"]:
            self.video_transform = VideoTransform(subset="test")

        self.modelmodule = ModelModule(cfg)
        self.modelmodule.cuda().eval()


    def forward(self, data_filename, lang):
        data_filename = os.path.abspath(data_filename)
        assert os.path.isfile(data_filename), f"data_filename: {data_filename} does not exist."

        if self.modality in ["audio", "audiovisual"]:
            try:
                audio, sample_rate = self.load_audio(data_filename)
            except:
                if os.path.isfile(data_filename.replace('.mp4', '.wav')):
                    audio_filename = data_filename.replace('.mp4', '.wav')
                    audio, sample_rate = self.load_audio(audio_filename)
                else:
                    raise ValueError(f"Unable to load audio for {data_filename}")
            audio = self.audio_process(audio, sample_rate)
            audio = audio.transpose(1, 0)
            audio = self.audio_transform(audio)

        if self.modality in ["video", "audiovisual"]:
            video = self.load_video(data_filename)
            video = torch.tensor(video)
            video = video.permute((0, 3, 1, 2))
            video = self.video_transform(video).cuda()

        if self.modality == "video":
            with torch.no_grad():
                transcript = self.modelmodule(video, lang=lang)
        elif self.modality == "audio":
            with torch.no_grad():
                transcript = self.modelmodule(audio)
        elif self.modality == "audiovisual":
            rate_ratio = len(audio) // len(video)
            if rate_ratio > 670 or rate_ratio < 530:
                print(f"WARNING: Inconsistent frame ratio for {data_filename}. Found audio length: {len(audio)}, video length: {len(video)}. It might affect the performance.")
            if rate_ratio != 640:
                audio = cut_or_pad(audio, len(video) * 640)
            with torch.no_grad():
                transcript = self.modelmodule(video, audio, lang=lang)

        return transcript


    def load_audio(self, data_filename: str, sr: int = 16000):
        """
        Sourced from https://github.com/openai/whisper/blob/main/whisper/audio.py
        Open an audio file and read as mono waveform, resampling as necessary

        Parameters
        ----------
        file: str
            The audio file to open

        sr: int
            The sample rate to resample the audio if necessary

        Returns
        -------
        A NumPy array containing the audio waveform, in float32 dtype.
        """

        # This launches a subprocess to decode audio while down-mixing
        # and resampling as necessary.  Requires the ffmpeg CLI in PATH.
        # fmt: off
        cmd = ["ffmpeg", "-nostdin", "-threads", "0", "-i", data_filename, "-f", "s16le", "-ac", "1", "-acodec", "pcm_s16le", "-ar", str(sr), "-"]
        # fmt: on
        try:
            out = run(cmd, capture_output=True, check=True).stdout
        except CalledProcessError as e:
            raise RuntimeError(f"Failed to load audio: {e.stderr.decode()}") from e

        waveform = np.frombuffer(out, np.int16).flatten().astype(np.float32) / 32768.0
        waveform = torch.FloatTensor(waveform).unsqueeze(0)

        return waveform, sr

    def load_video(self, data_filename):
        return torchvision.io.read_video(data_filename, pts_unit="sec")[0].numpy()

    def audio_process(self, waveform, sample_rate, target_sample_rate=16000):
        if sample_rate != target_sample_rate:
            waveform = torchaudio.functional.resample(
                waveform, sample_rate, target_sample_rate
            )
        waveform = torch.mean(waveform, dim=0, keepdim=True)
        return waveform

In [5]:
from hydra import compose, initialize
with initialize(version_base=None, config_path="conf", job_name="test_app"):
    cfg = compose(config_name="config")
gpu = 0
torch.cuda.set_device(gpu)
pipeline = InferencePipeline(cfg)
pipeline.modelmodule.cuda(gpu)
pipeline.modelmodule = torch.nn.DataParallel(pipeline.modelmodule, device_ids=[gpu])




In [6]:
from IPython.display import HTML
from base64 import b64encode

## play_video function based on: https://colab.research.google.com/drive/1bNXkfpHiVHzXQH8WjGhzQ-fsDxolpUjD

def play_video(video_path, width=200):
  mp4 = open(video_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML(f"""
  <video width={width} controls>
        <source src="{data_url}" type="video/mp4">
  </video>
  """)

In [7]:
import os
import cv2
import torchvision
from pipelines.data.data_module import AVSRDataLoader
from pipelines.detectors.mediapipe.detector import LandmarksDetector

def save2vid(filename, vid, frames_per_second):
    torchvision.io.write_video(filename, vid, frames_per_second)

def preprocess_video(src_filename, dst_filename):
    landmarks = landmarks_detector(src_filename)
    data = dataloader.load_data(src_filename, landmarks)
    fps = cv2.VideoCapture(src_filename).get(cv2.CAP_PROP_FPS)
    save2vid(dst_filename, data, fps)
    return

dataloader = AVSRDataLoader(modality="video", speed_rate=1, transform=False, detector="mediapipe", convert_gray=False)
landmarks_detector = LandmarksDetector()

In [8]:
preprocess_video(src_filename="clip.mp4", dst_filename="roi.mp4")
play_video("roi.mp4", width=300)

/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [10]:
transcript = pipeline("roi.mp4",2)
print(transcript)

completely our straight environments where we have large changes in catapults and


In [18]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.196.78:8501

npx: installed 22 in 1.915s
your url is: https://brown-onions-admire.loca.lt
2024-06-20 10:43:12.057549: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 10:43:12.057602: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 10:43:12.059061: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-20 10:43:13.302761: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin